In [ ]:
#Task 1

In [134]:
import pandas as pd
import random

In [135]:
#locate the file from computer path
gas_charge ='/Users/seunghokang/Downloads/gorilla_test/data.xlsx'

In [136]:
#data were extracted from each sheet and concatened with new index
forecast_sheet = pd.concat(pd.read_excel(gas_charge, sheet_name=['Forecasted Consumption']), ignore_index = True)
rate_sheet = pd.concat(pd.read_excel(gas_charge, sheet_name=['Rates']), ignore_index = True)
meter_sheet = pd.concat(pd.read_excel(gas_charge, sheet_name=['Meter List']), ignore_index = True)

In [141]:
#first we create a DF to store final result, 
#assign the array since there are three meter ID & three zones
#if this is not done, we 
Cost_table = pd.DataFrame({'Meter ID':[0,0,0],
                            'Total Estimated Consumption (kWh)':[0,0,0],
                                'Total Cost (£)':[0,0,0]})

# We need loop every values from meter list using the positional indexes [0 to n-1] 
# Because we cannot do 
for i in range(len(meter_sheet)):
    m_reading = meter_sheet.iloc[i] 

    # Compare Exit Zone and AQ with Rate data to find the correct meter reading for the given days:    
    # first we locate and extract exit zone (loc uses the column label) from the rate_sheet to match with exit zone of the correct meterID
    exit_zone = rate_sheet.loc[(rate_sheet['Exit Zone']) == m_reading['Exit Zone']]
    
    # second locate and extract data that have AQ(meter) between AQ(min) and AQ(MAX)
    AQ_band = rate_sheet.loc[(m_reading['Annual Quantity (kWh)'] > rate_sheet['Annual Quantity (Min)'])
                          & (m_reading['Annual Quantity (kWh)'] < rate_sheet['Annual Quantity (Max)'])]
    
    # Third locate and extract forecasted dates of exit zones and AQ_band
    pred_dates = rate_sheet.loc[(rate_sheet['Date'] >= '2020-09-30') & 
                          (rate_sheet["Date"] <= '2022-09-30')]
    
    
    # we are duplicating the code (bad practice, combined two DF)
    combined = rate_sheet.loc[(rate_sheet['Exit Zone'] == m_reading['Exit Zone']) &
                      (m_reading['Annual Quantity (kWh)'] > rate_sheet['Annual Quantity (Min)'])
                          & (m_reading['Annual Quantity (kWh)'] < rate_sheet['Annual Quantity (Max)']) &
                      (rate_sheet['Date'] >= '2020-10-1') & (rate_sheet['Date'] <= '2022-09-30')].index.values
    
    #print(combined)

    
    # now we create a empty table to calculate the rate
    # the date from our interest(i.e. forecast sheet)
    rate_cal = pd.DataFrame(columns = ('Date','Rate','Day Usuage','Cost per day'))
    rate_cal['Date'] = pd.date_range('2020-10-1', '2022-09-30')
    # print(rateCal)
    
    # first we loop the combined
    # the rate of daily usuage from rate_sheet will be matched with the date from the combined DF
    # second we get the date of rate sheet & combined df and match with newly create df
    # third once we get the date, we can add matched daily rate of gas from rate_sheet
    # note: we add substract 1 to prevent index out of range 
    for j in range(len(combined)-1):
        #print (j)
        rate_cal['Rate'].loc[((rate_cal['Date'] >= rate_sheet['Date'][combined[j]]) & 
                               (rate_cal['Date'] < rate_sheet['Date'][combined[j+1]])).values] \
                                = rate_sheet['Rate (p/kWh)']
    
    
    # now we can get meter IDs and their associated daily usuage of the meters    
    rate_cal['Daily use'] = forecast_sheet['kWh'].loc[(forecast_sheet['Meter ID'] == m_reading['Meter ID'])].values
    
    # we work out the cost per day
    rate_cal['Cost per day'] = rate_cal['Daily use'] * rate_cal['Rate']
    
    #Finally using the two variables and sum function, we can calculate Total 
    total_estimated_consumption = rate_cal['Daily use'].sum()
    total_cost = rate_cal['Cost per day'].sum() *0.01
    
    #assigning the values in the DF
    Cost_table.iloc[i] = [m_reading['Meter ID'], total_estimated_consumption, total_cost]

print(Cost_table)

     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0  1000000603                       24674.000000       22.833364
1    10588707                      122449.999999      115.327961
2    10626610                      303558.000002      274.531211


/Users/seunghokang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [143]:
#Task 2

In [ ]:
def random_meter(n=random, randint(0,100)):
    